#  Cash-Karp Runge-Kutta method with adaptive stepwise control

### Import necessary libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Define our coupled derivatives to integrate

In [ ]:
def dydx(x,y):
    
    #set the derivatives
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0] and z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = x
    y_derivs[0] = y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #return the array of dydx
    return y_derivs

### Define the Cash-Karp Runge-Kutta method estimate one (yn+1)

In [ ]:
def ck_rk_core1(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    x_temp = xi + (1/5)*h
    y_temp[:] = yi[:] + (1/5)*k1[:]
    y_derivs = dydx(x_temp,y_temp)
    k2[:] = h*y_derivs[:]
   
    #get k3 values
    x_temp = xi + (3/10)*h
    y_temp[:] = yi[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(x_temp,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    x_temp = xi + (3/5)*h
    y_temp[:] = yi[:] + (3/10)*k1[:] + (-9/10)*k2[:] + (6/5)*k3[:]
    y_derivs = dydx(x_temp,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    x_temp = xi + (1)*h
    y_temp[:] = yi[:] + (-11/54)*k1[:] + (5/2)*k2[:] + (-70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(x_temp,y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    x_temp = xi + (7/8)*h
    y_temp[:] = yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:]
    y_derivs = dydx(x_temp,y_temp)
    k6[:] = h*y_derivs[:]
    
    #caluclate first estimate yn+1
    ye1 = yi + (37/378)*k1[:] + (0)*k2[:] + (250/621)*k3[:] + (125/594)*k4[:] + (0)*k5[:] + (512/1771)*k6[:]
    
    #this is an array
    return ye1

### Define the Cash-Karp Runge-Kutta method estimate two (y*n+1)

In [ ]:
def ck_rk_core2(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    x_temp = xi + (1/5)*h
    y_temp[:] = yi[:] + (1/5)*k1[:]
    y_derivs = dydx(x_temp,y_temp)
    k2[:] = h*y_derivs[:]
   
    #get k3 values
    x_temp = xi + (3/10)*h
    y_temp[:] = yi[:] + (3/40)*k1[:] + (9/40)*k2[:]
    y_derivs = dydx(x_temp,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    x_temp = xi + (3/5)*h
    y_temp[:] = yi[:] + (3/10)*k1[:] + (-9/10)*k2[:] + (6/5)*k3[:]
    y_derivs = dydx(x_temp,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    x_temp = xi + (1)*h
    y_temp[:] = yi[:] + (-11/54)*k1[:] + (5/2)*k2[:] + (-70/27)*k3[:] + (35/27)*k4[:]
    y_derivs = dydx(x_temp,y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    x_temp = xi + (7/8)*h
    y_temp[:] = yi[:] + (1631/55296)*k1[:] + (175/512)*k2[:] + (575/13824)*k3[:] + (44275/110592)*k4[:] + (253/4096)*k5[:]
    y_derivs = dydx(x_temp,y_temp)
    k6[:] = h*y_derivs[:]
    
    #calculate second estimate y*n+1
    ye2 = yi + (2825/27648)*k1[:] + (0)*k2[:] + (18575/48384)*k3[:] + (13525/55296)*k4[:] + (277/14336)*k5[:] + (1/4)*k6[:]

    return ye2

### Define an adaptive step size driver for Cash-Karp Runge-Kutta method

In [ ]:
def ck_rk_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0 
    
    #create an error 
    Delta = np.array(nv)
    
    #remember the step
    h_step = h
    
    #adjust the step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error 
        ye1 = ck_rk_core1(dydx,x_i,y_i,nv,h_step)
        ye2 = ck_rk_core2(dydx,x_i,y_i,nv,h_step)
        
        #compute an error
        Delta = np.fabs(ye1 - ye2)
        
        #if the error is too latge, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check the iteration
        if(i>imax):
            print("Too many iterations in ck_rk_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i += 1 
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return ye1, h_new, h_step

### Define a wrapper for Cash-Karp Runge-Kutta

In [ ]:
def ck_rk(dydx,a,b,y_a,tol):
    
    #dydx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound 
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #sefine our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0 
    
    #set the number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1

    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = ck_rk_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = ck_rk_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
        
        #update values
        xi += h_step
        yi[:] = yi_new[:]
    
        #add the step to the arrays 
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
    
        #prevent too many iterations
        if(i>=imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i += 1
    
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i, xi, h_step, b)
        print(s)
    
        #break if new xi is == b
        if(xi==b):
            flag = 0

    #return the answer
    return x,y

### Perform the integration

In [ ]:
a = 0.0
b = 2.0 * np.pi 

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration 
x,y = ck_rk(dydx,a,b,y_0,tolerance)

### Plot the result

In [ ]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

### Plot the error

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)